# Model Evaluation - Combo Loss Model
Evaluates the 3D U-Net trained with Combo Loss (Dice + Focal + Tversky) on the test set.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# =============================================================================
# Define Loss Functions (needed for model loading)
# =============================================================================

# Class weights and parameters (must match training)
CLASS_WEIGHTS = [0.1, 1.0, 30.0]
TVERSKY_ALPHA = 0.3
TVERSKY_BETA = 0.7
FOCAL_GAMMA = 2.0
LOSS_WEIGHTS = {'dice': 1.0, 'focal': 1.0, 'tversky': 1.0}


def dice_coefficient_per_class(y_true, y_pred, class_idx, smooth=1e-6):
    """Dice coefficient for a single class."""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_c = y_true[..., class_idx]
    y_pred_c = y_pred[..., class_idx]
    y_true_f = tf.keras.backend.flatten(y_true_c)
    y_pred_f = tf.keras.backend.flatten(y_pred_c)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def weighted_dice_loss(y_true, y_pred):
    """Weighted dice loss."""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    total_loss = 0.0
    for class_idx, weight in enumerate(CLASS_WEIGHTS):
        dice = dice_coefficient_per_class(y_true, y_pred, class_idx)
        total_loss += weight * (1 - dice)
    return total_loss / sum(CLASS_WEIGHTS)


def focal_loss(y_true, y_pred, gamma=FOCAL_GAMMA):
    """Focal loss for handling class imbalance."""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    focal_weight = tf.pow(1 - y_pred, gamma)
    ce = -y_true * tf.math.log(y_pred)
    class_weights_tensor = tf.constant(CLASS_WEIGHTS, dtype=tf.float32)
    class_weights_tensor = class_weights_tensor / tf.reduce_sum(class_weights_tensor)
    focal = focal_weight * ce * class_weights_tensor
    return tf.reduce_mean(tf.reduce_sum(focal, axis=-1))


def tversky_loss(y_true, y_pred, alpha=TVERSKY_ALPHA, beta=TVERSKY_BETA, smooth=1e-6):
    """Tversky loss for controlling precision/recall tradeoff."""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    total_loss = 0.0
    for class_idx, weight in enumerate(CLASS_WEIGHTS):
        y_true_c = y_true[..., class_idx]
        y_pred_c = y_pred[..., class_idx]
        y_true_f = tf.keras.backend.flatten(y_true_c)
        y_pred_f = tf.keras.backend.flatten(y_pred_c)
        tp = tf.keras.backend.sum(y_true_f * y_pred_f)
        fp = tf.keras.backend.sum((1 - y_true_f) * y_pred_f)
        fn = tf.keras.backend.sum(y_true_f * (1 - y_pred_f))
        tversky_idx = (tp + smooth) / (tp + alpha * fp + beta * fn + smooth)
        total_loss += weight * (1 - tversky_idx)
    return total_loss / sum(CLASS_WEIGHTS)


def combo_loss(y_true, y_pred):
    """Combined loss: Dice + Focal + Tversky."""
    dice = weighted_dice_loss(y_true, y_pred)
    focal = focal_loss(y_true, y_pred)
    tversky = tversky_loss(y_true, y_pred)
    total = (LOSS_WEIGHTS['dice'] * dice + 
             LOSS_WEIGHTS['focal'] * focal + 
             LOSS_WEIGHTS['tversky'] * tversky)
    return total / sum(LOSS_WEIGHTS.values())


def dice_coefficient(y_true, y_pred):
    """Overall dice coefficient."""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1e-6) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1e-6)


def dice_liver(y_true, y_pred):
    return dice_coefficient_per_class(y_true, y_pred, 1)


def dice_tumor(y_true, y_pred):
    return dice_coefficient_per_class(y_true, y_pred, 2)


print("Loss functions defined.")

In [ ]:
# =============================================================================
# Setup Test Files (same split as training)
# =============================================================================

DATA_DIR = 'preprocessed_patches_v2'
NUM_CLASSES = 3
SEED = 42
BATCH_SIZE = 4

# Get test files (same split as training)
all_files = sorted([os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.npz')])
np.random.seed(SEED)
indices = np.random.permutation(len(all_files))
train_end = int(len(all_files) * 0.70)
val_end = train_end + int(len(all_files) * 0.15)
test_files = [all_files[i] for i in indices[val_end:]]

print(f"Total files: {len(all_files)}")
print(f"Test set: {len(test_files)} files ({len(test_files) * 20} patches)")

In [ ]:
# =============================================================================
# Load Combo Loss Model + Warmup
# =============================================================================

MODEL_PATH = 'checkpoints/best_model_combo_loss.keras'

print(f"Loading model from {MODEL_PATH}...")
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        'combo_loss': combo_loss,
        'weighted_dice_loss': weighted_dice_loss,
        'focal_loss': focal_loss,
        'tversky_loss': tversky_loss,
        'dice_coefficient': dice_coefficient,
        'dice_liver': dice_liver,
        'dice_tumor': dice_tumor
    }
)
print("Model loaded successfully!")
print(f"Model parameters: {model.count_params():,}")

# Warmup prediction to trigger XLA compilation
print("\nWarming up model (XLA compilation - this takes 1-2 min)...")
warmup_start = time.time()
dummy_input = np.zeros((1, 128, 128, 128, 1), dtype=np.float32)
_ = model.predict(dummy_input, verbose=0)
print(f"Warmup done in {time.time()-warmup_start:.1f}s - ready for evaluation!")

In [ ]:
# =============================================================================
# Compute Confusion Matrix
# =============================================================================

print("Computing confusion matrix on test set...")
print(f"Processing {len(test_files)} files ({len(test_files)*20} patches)...\n")

cm = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.int64)
total_patches = 0
start_time = time.time()

for file_idx, filepath in enumerate(test_files):
    file_start = time.time()
    data = np.load(filepath)
    patches = data['patches'].astype(np.float32) / 255.0
    segs = data['segmentations']
    
    # Process in batches
    for i in range(0, len(patches), BATCH_SIZE):
        x = patches[i:i+BATCH_SIZE][..., np.newaxis]
        y_true = segs[i:i+BATCH_SIZE]
        
        pred = model.predict(x, verbose=0)
        y_pred = np.argmax(pred, axis=-1)
        
        # Accumulate confusion matrix
        cm += confusion_matrix(y_true.flatten(), y_pred.flatten(), labels=[0, 1, 2])
        total_patches += len(x)
    
    file_time = time.time() - file_start
    total_time = time.time() - start_time
    eta = (total_time / (file_idx + 1)) * (len(test_files) - file_idx - 1)
    print(f"  [{file_idx+1:2d}/{len(test_files)}] {os.path.basename(filepath)}: {file_time:.1f}s (ETA: {eta:.0f}s)")

print(f"\n{'='*50}")
print(f"DONE! Total time: {time.time()-start_time:.1f}s")
print(f"Total voxels evaluated: {cm.sum():,}")

In [ ]:
# =============================================================================
# Plot Confusion Matrix
# =============================================================================

class_names = ['Background', 'Liver', 'Tumor']
cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
im1 = axes[0].imshow(cm, cmap='Blues')
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14)
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('True', fontsize=12)
axes[0].set_xticks(range(3))
axes[0].set_yticks(range(3))
axes[0].set_xticklabels(class_names)
axes[0].set_yticklabels(class_names)
for i in range(3):
    for j in range(3):
        axes[0].text(j, i, f'{cm[i,j]:,}', ha='center', va='center', fontsize=10)
plt.colorbar(im1, ax=axes[0])

# Normalized (percentages)
im2 = axes[1].imshow(cm_norm, cmap='Blues', vmin=0, vmax=100)
axes[1].set_title('Confusion Matrix (% by True Class)', fontsize=14)
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('True', fontsize=12)
axes[1].set_xticks(range(3))
axes[1].set_yticks(range(3))
axes[1].set_xticklabels(class_names)
axes[1].set_yticklabels(class_names)
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{cm_norm[i,j]:.1f}%', ha='center', va='center', fontsize=10)
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig('confusion_matrix_combo.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nConfusion matrix saved to confusion_matrix_combo.png")

In [ ]:
# =============================================================================
# Print Per-Class Metrics
# =============================================================================

print("="*60)
print("PER-CLASS METRICS (Combo Loss Model)")
print("="*60)
print(f"{'Class':<12} {'Precision':>10} {'Recall':>10} {'F1':>10} {'Dice':>10}")
print("-"*60)

metrics_dict = {}
for i, name in enumerate(class_names):
    tp = cm[i, i]
    fp = cm[:, i].sum() - tp
    fn = cm[i, :].sum() - tp
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    
    metrics_dict[name] = {'precision': precision, 'recall': recall, 'f1': f1, 'dice': dice}
    print(f"{name:<12} {precision:>10.4f} {recall:>10.4f} {f1:>10.4f} {dice:>10.4f}")

print("="*60)

# Overall accuracy
accuracy = np.trace(cm) / cm.sum()
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# =============================================================================
# Compare with Baseline (if available)
# =============================================================================

print("\n" + "="*60)
print("COMPARISON WITH BASELINE")
print("="*60)

# Baseline metrics from original model (best_model_v2_augment.keras)
baseline = {
    'Tumor': {'recall': 0.527, 'dice': 0.65},  # From training history
    'Liver': {'recall': 0.90, 'dice': 0.90}
}

print(f"\n{'Metric':<20} {'Baseline':>12} {'Combo Loss':>12} {'Change':>12}")
print("-"*60)

# Tumor metrics
tumor_recall_change = metrics_dict['Tumor']['recall'] - baseline['Tumor']['recall']
tumor_dice_change = metrics_dict['Tumor']['dice'] - baseline['Tumor']['dice']
print(f"{'Tumor Recall':<20} {baseline['Tumor']['recall']:>12.4f} {metrics_dict['Tumor']['recall']:>12.4f} {tumor_recall_change:>+12.4f}")
print(f"{'Tumor Dice':<20} {baseline['Tumor']['dice']:>12.4f} {metrics_dict['Tumor']['dice']:>12.4f} {tumor_dice_change:>+12.4f}")

# Liver metrics
liver_recall_change = metrics_dict['Liver']['recall'] - baseline['Liver']['recall']
liver_dice_change = metrics_dict['Liver']['dice'] - baseline['Liver']['dice']
print(f"{'Liver Recall':<20} {baseline['Liver']['recall']:>12.4f} {metrics_dict['Liver']['recall']:>12.4f} {liver_recall_change:>+12.4f}")
print(f"{'Liver Dice':<20} {baseline['Liver']['dice']:>12.4f} {metrics_dict['Liver']['dice']:>12.4f} {liver_dice_change:>+12.4f}")

print("\n" + "="*60)
if tumor_recall_change > 0:
    print(f"SUCCESS: Tumor recall improved by {tumor_recall_change*100:.1f}%!")
else:
    print(f"Tumor recall decreased by {abs(tumor_recall_change)*100:.1f}%")
print("="*60)

In [ ]:
# =============================================================================
# Visualize Sample Predictions
# =============================================================================

def visualize_prediction(model, filepath, patch_idx=0, slice_idx=64):
    """Visualize a single prediction with ground truth."""
    data = np.load(filepath)
    patch = data['patches'][patch_idx].astype(np.float32) / 255.0
    seg_true = data['segmentations'][patch_idx]
    
    # Predict
    pred = model.predict(patch[np.newaxis, ..., np.newaxis], verbose=0)
    seg_pred = np.argmax(pred[0], axis=-1)
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(patch[:, :, slice_idx], cmap='gray')
    axes[0].set_title('CT Slice')
    axes[0].axis('off')
    
    axes[1].imshow(seg_true[:, :, slice_idx], cmap='jet', vmin=0, vmax=2)
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')
    
    axes[2].imshow(seg_pred[:, :, slice_idx], cmap='jet', vmin=0, vmax=2)
    axes[2].set_title('Prediction')
    axes[2].axis('off')
    
    plt.suptitle(f'{os.path.basename(filepath)} - Patch {patch_idx}, Slice {slice_idx}')
    plt.tight_layout()
    plt.show()

# Visualize a few samples
print("Sample predictions from test set:")
for i in range(min(3, len(test_files))):
    visualize_prediction(model, test_files[i], patch_idx=0, slice_idx=64)